In [2]:
import os
from dotenv import load_dotenv

In [3]:
load_dotenv()
os.environ['GROQ_API_KEY']=os.getenv('GROQ_API_KEY')

In [5]:
from langchain_groq import ChatGroq

In [6]:
model=ChatGroq(model="gemma2-9b-it")


In [7]:
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000022EBD87BFB0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000022EBD8AD0D0>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [8]:
model.invoke("hello")

AIMessage(content='Hello! 👋  How can I help you today?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 10, 'total_tokens': 24, 'completion_time': 0.025454545, 'prompt_time': 4.3e-07, 'queue_time': 0.020612576, 'total_time': 0.025454975}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-a2179645-94f9-4c4a-96e1-176b65157b2f-0', usage_metadata={'input_tokens': 10, 'output_tokens': 14, 'total_tokens': 24})

### exploring message history

In [12]:
from langchain_core.messages import HumanMessage, AIMessage

In [22]:
model.invoke(
    [
        HumanMessage("my name is HV and i am data scientist"),
        AIMessage(content="Nice to meet you, HV!  \n\nIt's great to know you're a data scientist. What kind of work do you do? \n\nI'm always interested in learning more about how people use data to solve problems and make discoveries.\n"),
        HumanMessage("i work on generative ai, nlp and forecasting. Whats my name and what do i do?"),
        ]
    )

AIMessage(content="You are HV, a data scientist who specializes in generative AI, NLP (Natural Language Processing), and forecasting!  \n\nThat's a really exciting field to be in.  Do you have a favorite project you've worked on that combines any of those areas?  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 101, 'total_tokens': 160, 'completion_time': 0.107272727, 'prompt_time': 0.003323459, 'queue_time': 0.025445916, 'total_time': 0.110596186}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-455e5931-f8d1-4cc5-8801-963cd5c53e3f-0', usage_metadata={'input_tokens': 101, 'output_tokens': 59, 'total_tokens': 160})

### implementing message history

In [23]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [24]:
#cache 
store={}

def get_session_history(session_id:str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model, get_session_history)
with_message_history


RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableLambda(_enter_history), kwargs={}, config={'run_name': 'load_history'}, config_factories=[])
| RunnableBinding(bound=RunnableLambda(_call_runnable_sync), kwargs={}, config={'run_name': 'check_sync_or_async'}, config_factories=[]), kwargs={}, config={'run_name': 'RunnableWithMessageHistory'}, config_factories=[]), kwargs={}, config={}, config_factories=[], get_session_history=<function get_session_history at 0x0000022ECDC5ED40>, history_factory_config=[ConfigurableFieldSpec(id='session_id', annotation=<class 'str'>, name='Session ID', description='Unique identifier for a session.', default='', is_shared=True, dependencies=None)])

In [25]:
config={"configurable":{"session_id":'HV'}}

response=with_message_history.invoke(
    [HumanMessage(content="what is my name?")],
    config=config
)

print(response)

content="As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to learn it! 😊\n" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 14, 'total_tokens': 53, 'completion_time': 0.070909091, 'prompt_time': 7.498e-05, 'queue_time': 0.022772846000000003, 'total_time': 0.070984071}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None} id='run-72fb7208-8495-4207-bd8c-d6127741c7f2-0' usage_metadata={'input_tokens': 14, 'output_tokens': 39, 'total_tokens': 53}


In [26]:
get_session_history("HV")

InMemoryChatMessageHistory(messages=[HumanMessage(content='what is my name?', additional_kwargs={}, response_metadata={}), AIMessage(content="As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to learn it! 😊\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 14, 'total_tokens': 53, 'completion_time': 0.070909091, 'prompt_time': 7.498e-05, 'queue_time': 0.022772846000000003, 'total_time': 0.070984071}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-72fb7208-8495-4207-bd8c-d6127741c7f2-0', usage_metadata={'input_tokens': 14, 'output_tokens': 39, 'total_tokens': 53})])

### retrieve your history

In [31]:
config2={"configurable":{"session_id":'JC'}}

response=with_message_history.invoke(
    [HumanMessage(content="my name is John Cena and i like wrestling")],
    config=config2
)

print(response)

content="Well, hello there, John Cena! It's an honor to meet you.  \n\n 😉  \n\nYou and me both on liking wrestling! It's an awesome sport full of action, drama, and larger-than-life characters.  Who are some of your favorite wrestlers?  \n\nMaybe we can even talk about some epic matches you've seen!\n" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 78, 'prompt_tokens': 417, 'total_tokens': 495, 'completion_time': 0.141818182, 'prompt_time': 0.022047216, 'queue_time': 0.02321231, 'total_time': 0.163865398}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None} id='run-9e6405dd-f7cd-4815-ba4e-6109af4b3652-0' usage_metadata={'input_tokens': 417, 'output_tokens': 78, 'total_tokens': 495}


In [33]:
response=with_message_history.invoke(
    [HumanMessage(content="what is my name")],
    config=config2
)
print(response)

content="You told me your name is John Cena!  \n\n Are you ready to discuss some wrestling?  I'm excited to hear your thoughts on the world of sports entertainment!  \n\n\n\n" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 507, 'total_tokens': 546, 'completion_time': 0.070909091, 'prompt_time': 0.016177003, 'queue_time': 0.023612479000000002, 'total_time': 0.087086094}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None} id='run-b6e95157-5939-4ad6-b701-40a9b4627278-0' usage_metadata={'input_tokens': 507, 'output_tokens': 39, 'total_tokens': 546}


### Prompt template

In [36]:
from  langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt=ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all the questions to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

In [37]:
chain=prompt|model

In [38]:
chain.invoke({"messages":[HumanMessage(content="hi my name is HV")]})

AIMessage(content="Hi HV, it's nice to meet you!  \n\nI'm ready to help with any questions you have.  What can I do for you today? 😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 30, 'total_tokens': 69, 'completion_time': 0.070909091, 'prompt_time': 0.00013747, 'queue_time': 0.021839108, 'total_time': 0.071046561}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-9bbcba01-80cd-4d34-8f91-30185ab8e569-0', usage_metadata={'input_tokens': 30, 'output_tokens': 39, 'total_tokens': 69})

In [39]:
with_message_history=RunnableWithMessageHistory(chain, get_session_history)

In [40]:
config3={"configurable":{"session_id":'JW'}}

response=with_message_history.invoke(
    [HumanMessage(content="my name is John Wick and i like action movies")],
    config=config3
)

print(response)

content="Hello John Wick! It's nice to meet you.  \n\nAs a helpful assistant, I'm here to answer your questions and chat about anything you like.  \n\nSince you enjoy action movies, I can:\n\n* **Recommend some action movies you might like**, based on your favorites. Just tell me what you've enjoyed in the past!\n* **Discuss specific action movie tropes or genres** you're interested in. Do you prefer martial arts films, gun-fu, explosions, or something else entirely?\n* **Help you find information about your favorite action movies**, like release dates, cast lists, or trivia.\n\nWhat can I do for you first, John Wick? \n\n" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 148, 'prompt_tokens': 35, 'total_tokens': 183, 'completion_time': 0.269090909, 'prompt_time': 0.000305959, 'queue_time': 0.069165928, 'total_time': 0.269396868}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None} id='run-c0aa5b69-

In [41]:
response=with_message_history.invoke(
    [HumanMessage(content="what is my name and what do i like")],
    config=config3
)

print(response)

content="You told me your name is John Wick and you like action movies!  \n\nIs there anything else you'd like to talk about, John Wick? Perhaps a particular action movie you've been wanting to discuss? 😄  \n" additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 199, 'total_tokens': 248, 'completion_time': 0.089090909, 'prompt_time': 0.007222341, 'queue_time': 0.021712174, 'total_time': 0.09631325}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None} id='run-00a086ce-8b50-49c3-bd3e-a62eff7f5fe9-0' usage_metadata={'input_tokens': 199, 'output_tokens': 49, 'total_tokens': 248}


### adding languages

In [44]:
prompt=ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all the questions to the best of your ability in the language {language}"),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain=prompt|model

In [47]:
response = chain.invoke({"messages":[HumanMessage(content="what is my name and what do i like")], "language":"Hindi"})
print(response)

content='मुझे आपके नाम और पसंद के बारे में जानकारी नहीं है क्योंकि मैं एक भाषा मॉडल हूँ और मेरे पास कोई याददाश्त या व्यक्तिगत जानकारी नहीं होती। \n\nआपका नाम क्या है और आपको क्या पसंद है, यह बताएं तो मैं आपसे बेहतर तरीके से बात कर सकूँगा। \n\n' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 77, 'prompt_tokens': 38, 'total_tokens': 115, 'completion_time': 0.14, 'prompt_time': 0.000301199, 'queue_time': 0.022759879, 'total_time': 0.140301199}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None} id='run-36944d31-31aa-412e-bc4b-f533c84ce5bf-0' usage_metadata={'input_tokens': 38, 'output_tokens': 77, 'total_tokens': 115}


### trimming chat history

In [51]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer=trim_messages(
    max_tokens=60,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

In [52]:
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

c:\Users\Harsh Vora\OneDrive\Desktop\PROJECTS\agenticai\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Harsh Vora\OneDrive\Desktop\PROJECTS\agenticai\venv\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Harsh Vora\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an adm

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [ ]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = RunnablePassthrough.assign(#####fwefwedwqqwdqwqwdq=itemgetter('messages')| trimmer)|prompt|model

In [57]:
chain

RunnableAssign(mapper={
  messages: RunnableLambda(itemgetter('messages'))
            | RunnableLambda(...)
})
| ChatPromptTemplate(input_variables=['language', 'messages'], input_types={'messages': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.An

In [60]:
response=chain.invoke(
    {
        "messages":messages+[HumanMessage(content="what icecream do i like?")],
        "language":"English"
    }
)

In [61]:
print(response)

content='You said you like vanilla ice cream!  🍦 😊  Is that still your favorite? \n' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 105, 'total_tokens': 127, 'completion_time': 0.04, 'prompt_time': 0.003698001, 'queue_time': 0.021340646, 'total_time': 0.043698001}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None} id='run-b3bf4d3f-951a-43cf-a948-50c89894c666-0' usage_metadata={'input_tokens': 105, 'output_tokens': 22, 'total_tokens': 127}
